In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from datetime import datetime
import os

transactions_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks\chunk_1_20250624_202535.csv")
customer_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\customer data\customer data.csv")
transactions_df.head()
customer_df.head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0


In [4]:
transactions_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [5]:
customer_df.head()

,Source,Target,Weight,typeTrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0


In [7]:
merged_df = transactions_df.merge(
    customer_df,
    how='inner',
    left_on=['customer', 'merchant', 'category','amount'],
    right_on=['Source', 'Target', 'typeTrans','Weight']
)
merged_df

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud_x,Source,Target,Weight,typeTrans,fraud_y
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0,'C1093826151','M348934600',4.55,'es_transportation',0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0,'C352968107','M348934600',39.68,'es_transportation',0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0,'C2054744914','M1823072687',26.89,'es_transportation',0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0,'C1760612790','M348934600',17.25,'es_transportation',0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0,'C757503768','M348934600',35.72,'es_transportation',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10099,4,'C1090528803','3','F','28007','M151143676','28007','es_barsandrestaurants',75.05,0,'C1090528803','M151143676',75.05,'es_barsandrestaurants',0
10100,4,'C567616900','4','M','28007','M1823072687','28007','es_transportation',12.94,0,'C567616900','M1823072687',12.94,'es_transportation',0
10101,4,'C2115344024','3','M','28007','M855959430','28007','es_hyper',8.45,0,'C2115344024','M855959430',8.45,'es_hyper',0
10102,4,'C1232356156','4','F','28007','M348934600','28007','es_transportation',5.65,0,'C1232356156','M348934600',5.65,'es_transportation',0


In [8]:
# Merchant-level transaction count
merchant_txn_counts = merged_df.groupby('merchant')['customer'].count().reset_index(name='txn_count')

# Only keep merchants with >50K transactions
eligible_merchants = merchant_txn_counts[merchant_txn_counts['txn_count'] > 50000]['merchant']

# Work only on eligible merchants
df_eligible = merged_df[merged_df['merchant'].isin(eligible_merchants)]

# Compute avg weight per customer per merchant
avg_weight = df_eligible.groupby(['merchant', 'customer'])['Weight'].mean().reset_index()

# Compute threshold per merchant
weight_threshold = avg_weight.groupby('merchant')['Weight'].quantile(0.10).reset_index()
weight_threshold.rename(columns={'Weight': 'weight_thresh'}, inplace=True)

txn_count_per_customer = df_eligible.groupby(['merchant', 'customer']).size().reset_index(name='txn_count')
txn_count_threshold = txn_count_per_customer.groupby('merchant')['txn_count'].quantile(0.90).reset_index()
txn_count_threshold.rename(columns={'txn_count': 'txn_thresh'}, inplace=True)

# Join thresholds
df1 = avg_weight.merge(weight_threshold, on='merchant')
df1 = df1.merge(txn_count_per_customer, on=['merchant', 'customer'])
df1 = df1.merge(txn_count_threshold, on='merchant')

# Apply pattern filter
pattern1 = df1[
    (df1['Weight'] <= df1['weight_thresh']) & 
    (df1['txn_count'] >= df1['txn_thresh'])
]

pattern1['patternId'] = 'PatId1'
pattern1['actionType'] = 'UPGRADE'
pattern1['YStartTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
pattern1['detectionTime'] = pattern1['YStartTime']
pattern1['MerchantId'] = pattern1['merchant']
pattern1['customerName'] = pattern1['customer']